In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../../src')

# 小数点以下 桁数 6
pd.options.display.float_format = '{:.2f}'.format


In [2]:
import initial_condition


In [3]:
global data_dir, output_dir
data_dir = '../../data/interim/internal/filtered_before_agg/'
output_dir = '../../data/interim/internal/filter_after_agg/'


In [4]:
## 初期条件
ar = initial_condition.AR
year_style = initial_condition.YEAR_STYLE

year_start = initial_condition.YEAR_START
year_end = initial_condition.YEAR_END
year_range = initial_condition.YEAR_RANGE

extract_population = initial_condition.EXTRACT_POPULATION
top_p_or_num = initial_condition.TOP_P_OR_NUM


In [5]:
# 全体
all_df = pd.read_csv(f'{data_dir}addedclassification.csv', 
                     encoding='utf-8', 
                     sep=',', 
                     usecols=['reg_num', 
                              'right_person_name', 
                              f'{ar}_{year_style}'], 
                     dtype={'reg_num': str, 
                            'right_person_name': str, 
                            f'{ar}_{year_style}': np.int64})
all_df = all_df[all_df[f'{ar}_{year_style}'].isin(range(year_start, year_end+1))]\
                .drop_duplicates()
all_df[f'{ar}_{year_style}_period'] = f'{year_start}-{year_end}'

# 各期間
# sep_year_df_dict = {}
sep_year_df_list = []

for year in range(year_start, year_end+1, year_range):
    # sep_year_df_dict[f'{year}-{year+year_range-1}'] = all_df[all_df[f'{ar}_year'].isin(range(year, year+year_range))]
    sep_year_df = all_df[all_df[f'{ar}_{year_style}'].isin(range(year, year+year_range))]\
                        .drop_duplicates()
    sep_year_df[f'{ar}_{year_style}_period'] = f'{year}-{year+year_range-1}'
    sep_year_df_list.append(sep_year_df)


In [6]:
all_applicant_weight_df = all_df.groupby([f'{ar}_{year_style}_period', 'reg_num'])[['right_person_name']].nunique()\
                                .reset_index(drop=False).rename(columns={'right_person_name': 'applicant_weight'}).copy()
all_reg_num_df = pd.merge(all_df.drop(columns=[f'{ar}_{year_style}']), all_applicant_weight_df, 
                          on=[f'{ar}_{year_style}_period', 'reg_num'], how='left')
all_reg_num_df['reg_num'] = round(1 / all_reg_num_df['applicant_weight'], 2)
# display(all_reg_num_df.head())
all_reg_num_df = all_reg_num_df.drop(columns=['applicant_weight'])\
                               .groupby([f'{ar}_{year_style}_period', 'right_person_name'])[['reg_num']].sum().reset_index()
all_reg_num_df = all_reg_num_df.sort_values(by=['reg_num'], ascending=[False])\
                                 .reset_index(drop=True)
all_reg_num_df


,app_year_period,right_person_name,reg_num
0,1981-2010,パナソニツクホールデイングス株式会社,126129.97
1,1981-2010,キヤノン株式会社,73862.02
2,1981-2010,株式会社東芝,61299.73
3,1981-2010,日本電気株式会社,59818.48
4,1981-2010,三菱電機株式会社,59192.34
...,...,...,...
64325,1981-2010,日本電計株式会社,0.07
64326,1981-2010,東名開発株式会社,0.06
64327,1981-2010,東日本コンクリート株式会社,0.06
64328,1981-2010,建設ボーリング株式会社,0.06


In [7]:
sep_year_reg_num_df_list = []
for sep_year_df in sep_year_df_list:
    sep_year_applicant_weight_df = sep_year_df.groupby([f'{ar}_{year_style}_period', 'reg_num'])[['right_person_name']].nunique()\
                                              .reset_index(drop=False).rename(columns={'right_person_name': 'applicant_weight'}).copy()
    sep_year_reg_num_df = pd.merge(sep_year_df.drop(columns=[f'{ar}_{year_style}']), sep_year_applicant_weight_df, 
                          on=[f'{ar}_{year_style}_period', 'reg_num'], how='left')
    sep_year_reg_num_df['reg_num'] = round(1 / sep_year_reg_num_df['applicant_weight'], 2)
    # display(sep_year_reg_num_df.head())
    sep_year_reg_num_df = sep_year_reg_num_df.drop(columns=['applicant_weight'])\
                                .groupby([f'{ar}_{year_style}_period', 'right_person_name'])[['reg_num']].sum().reset_index()
    sep_year_reg_num_df = sep_year_reg_num_df.sort_values(by=['reg_num'], ascending=[False])\
                                    .reset_index(drop=True)
    sep_year_reg_num_df_list.append(sep_year_reg_num_df)

# sep_year_reg_num_df = pd.concat(sep_year_reg_num_df_list, axis='index', ignore_index=True)
# sep_year_reg_num_df


In [8]:
if extract_population == 'all':
    if top_p_or_num[0] == 'p': top = (all_reg_num_df['right_person_name'].nunique()*top_p_or_num[1])//100
    elif top_p_or_num[0] == 'num': top = top_p_or_num[1]

    all_reg_num_top_df = all_reg_num_df.head(top)
    all_reg_num_top_df
    all_reg_num_top_df.to_csv(
        f'{output_dir}{ar}_{year_style}_{extract_population}_reg_num_top_{top_p_or_num[0]}_{top_p_or_num[1]}.csv',
        encoding='utf-8', 
        sep=',', 
        index=False
    )
elif extract_population == 'sep_year':
    sep_year_reg_num_top_df_list = []
    for sep_year_reg_num_df in sep_year_reg_num_df_list:
        if top_p_or_num[0] == 'p': top = (sep_year_reg_num_df['right_person_name'].nunique()*top_p_or_num[1])//100
        elif top_p_or_num[0] == 'num': top = top_p_or_num[1]
        sep_year_reg_num_top_df_list.append(sep_year_reg_num_df.head(top))
    sep_year_reg_num_top_df = pd.concat(sep_year_reg_num_top_df_list, axis='index', ignore_index=True)
    sep_year_reg_num_top_df.to_csv(
        f'{output_dir}{ar}_{year_style}_{extract_population}_reg_num_top_{top_p_or_num[0]}_{top_p_or_num[1]}.csv',
        encoding='utf-8', 
        sep=',', 
        index=False
    )